In [1]:
# Copyright 2024 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

LLM Classification Transients
==============

<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/turanbulmus/spacehack/blob/main/01 - LLM Classification Transients.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo"><br> Open in Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2Fturanbulmus%2Fspacehack%2Fmain%2F01%20-%20LLM%20Classification%20Transients.ipynb">
      <img width="32px" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" alt="Google Cloud Colab Enterprise logo"><br> Open in Colab Enterprise
    </a>
  </td>    
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/turanbulmus/spacekhack/main/01%20-%20LLM%20Classification%20Transients.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo"><br> Open in Workbench
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/turanbulmus/spacehack/blob/main/01%20-%20LLM%20Classification%20Transients.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo"><br> View on GitHub
    </a>
  </td>
</table>

| | |
|----------|-------------|
| Authors   | Turan Bulmus,  Fiorenzo Stoppa|
| Last updated | 2025 03 20: Final Publication |



# Overview

This notebook showcases the potential of multimodal models like Gemini 1.5 Pro for tackling few-shot object classification problems. We'll explore how this advanced model can analyze telescope images, compare them against reference and difference images, and classify them as "real" or "bogus" solely based on visual information and human-provided instructions. By the end of this notebook, you'll gain insights into:

* How Gemini 1.5 Pro can perform few-shot classification tasks without explicit training.
* How Gemini 1.5 Pro not only classifies images but also provides human readable explanations for the classifications.
* The power of multimodal models to understand and reason about visual content.
* Building effective prompts for complex image analysis tasks.

TODO: Add link to the paper

This notebook demonstrates these with the following steps:

1. **Import Libraries and Build Functions:** We start by importing the necessary libraries for image loading, processing, and evaluation. We also define functions to facilitate the analysis and visualization of results.
2. **Build System Instructions for the Prompt:** A well-crafted prompt is crucial for guiding Gemini 1.5 Pro towards the desired output. We carefully define instructions outlining the task, the type of images, and the desired classification outcome.
3. **Load the Dataset:** We load a collection of telescope images, including reference, difference, and new images, to be used for the classification task.
4. **Run Gemini 1.5 Pro with the Prompt:** We execute the prompt with Gemini 1.5 Pro, iterating over 3000+ samples from the dataset to get classifications for each image.
5. **Evaluate Model Performance:** We evaluate the model's performance using metrics like a confusion matrix, precision, recall, and accuracy. These metrics provide insights into the model's ability to correctly classify "real" and "bogus" images. Finally, we visualize the results for better understanding.


## Using This Notebook

Colab is recommended for running this notebook, but it can run in any iPython environment where you can connect to Google Cloud, install pip packages, etc.

If you're running outside of Colab, depending on your environment you may need to install pip packages (at the very least `pandas` and `tabulate`) that are included in the Colab environment by default but are not part of the Python Standard Library--try pipping the library name of any imports that fail. You'll also notice some comments in code cells that look like "@something"; these have special rendering in colab, but you aren't missing out on any content or important functionality.

This notebook uses the following Google Cloud services and resources:

* [Vertex AI Gemini Models](https://cloud.google.com/vertex-ai/generative-ai/docs/learn/models)
* [Vertex AI Experiments](https://cloud.google.com/vertex-ai/docs/experiments/intro-vertex-ai-experiments)
* [Google Cloud BigQuery](https://cloud.google.com/bigquery/docs/introduction)

This notebook has been tested in the following environment:

* Python version = 3.12.3
* [google-cloud-aiplatform](https://pypi.org/project/google-cloud-aiplatform/) version = 1.70.0

## Useful Tips

1. This notebook uses Generative AI cababilities. Re-running a cell that uses Generative AI capabilities may produce similar but not identical results.

TODO: Add tips regarding: Quotas, possible errors (429 etc..)

# 0. Getting Started

## Install required packages and restart runtime 
Install the required packages for code to run. These packages are included in the requirements.txt file.

To use the newly installed packages in this Jupyter runtime, you must restart the runtime. You can do this by running the cell below, which restarts the current kernel.

The restart might take a minute or longer. After it's restarted, continue to the next step.

The code assumes that the packages are already installed hence the following code is commented out. If you are running this notebook for the first time, please uncomment the next block and run the code

In [2]:
# ! pip install -r requirements.txt --quiet

# import IPython

# app = IPython.Application.instance()
# app.kernel.do_shutdown(True)

<div class="alert alert-block alert-warning">
<b>⚠️ The kernel is going to restart. Please wait until it is finished before continuing to the next step. ⚠️</b>
</div>

## Authenticate your notebook environment

If you're using Colab, run the code in the next cell. Follow the popups and authenticate with an account that has access to your Google Cloud [project](https://cloud.google.com/resource-manager/docs/creating-managing-projects#identifying_projects).

If you're running this notebook somewhere besides Colab, make sure your environment has the right Google Cloud access. If that's a new concept to you, consider looking into [Application Default Credentials for your local environment](https://cloud.google.com/docs/authentication/provide-credentials-adc#local-dev) and [initializing the Google Cloud CLI](https://cloud.google.com/docs/authentication/gcloud). In many cases, running `gcloud auth application-default login` in a shell on the machine running the notebook kernel is sufficient.

More authentication options are discussed [here](https://cloud.google.com/docs/authentication).

In [3]:
# Colab authentication
import sys

if "google.colab" in sys.modules:
    from google.colab import auth

    auth.authenticate_user()
    print('Authenticated')

## Set Google Cloud environment variables information and initialize Clients

Learn more about [setting up a project and a development environment](https://cloud.google.com/vertex-ai/docs/start/cloud-environment).
To get started using Vertex AI, you must have an existing Google Cloud project and the following:
* [Enable the Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com). 
* [Enable the Big Query API](https://console.cloud.google.com/flows/enableapi?apiid=bigquery.googleapis.com).

The following variables will be used throughout the code below. Please find the meaning of each variable in the documentation below:

* [PROJECT_ID](https://cloud.google.com/resource-manager/docs/creating-managing-projects): The ID of the GCP project which will host the 
* [LOCATION](https://cloud.google.com/about/locations): The location where the computational resources will be run. Please also check the availability of Gemini models in the associated regions from [here](https://cloud.google.com/vertex-ai/generative-ai/docs/learn/locations).
* [EXPERIMENT_NAME](https://cloud.google.com/vertex-ai/docs/experiments/create-manage-exp-run): The name of the Vertex AI experiment. This will ensure tracebility of the experiement in the Vertex AI Environment
* [DATASET_ID](https://cloud.google.com/bigquery/docs/datasets-intro): The code will run a batch process for various LLM calls. The results and the input of these runs will be stored at a BigQuery table. The instructions to create a dataset can be found [here](https://cloud.google.com/bigquery/docs/datasets).

In [4]:
PROJECT_ID = "<YOUR_PROJECT_ID>"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}
EXPERIMENT_NAME = "supernovadetection" # @param {type:"string"}
# Make sure that dataset is created in Big Query
DATASET_ID = "spacehack" # @param {type:"string"}

import vertexai
from google.cloud import bigquery

# Initiate Vertex AI and BigQuery clients
vertexai.init(project=PROJECT_ID, location=LOCATION, experiment=EXPERIMENT_NAME)
bq_client = bigquery.Client(project=PROJECT_ID)

## Import libraries and define helper functions

Import the necessary libraries that will be used throughout the code. Please note that the last import (i.e. helper_functions) is coming originated from the source code in the repository.

In [5]:
import gdown 
import os, datetime

import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

from google.cloud import bigquery
import google.cloud.aiplatform as aiplatform

from helper_functions import build_run_batch, if_tbl_exists, create_ex, save_picture, save_prompt, build_experiment_vars

# 1. Build System Instructions and The Prompt

## Build the System Instructions
System instructions for an LLM (Large Language Model) are a set of instructions given at the very beginning of a conversation (or as a persistent setting) that tell the model how to behave, what role to assume, and what constraints to operate under.

Think of it as the LLM's "mission statement" for the entire interaction. They can:

* Define the LLM's persona: "You are a helpful and friendly chatbot."
* Set response style: "Answer concisely and use bullet points."
* Specify the task: "You are an expert in data science and will help users with their coding problems."
* Impose limitations: "Do not provide medical advice."
* Provide context: "The user is a beginner in Python programming."

Since one can not include images to system instructions (yet) below will only consist of text portion of the system instructions. 
For this project we are introducing four main parts to the System instructions:

1) Persona: Highlighting the persona of the model. Eg. "You are an experienced astrophysicist..."
2) Instructions: This is set of instructions that are also used as guidelines for human classifiers.
3) Task: Explaining the task that the LLM model needs to conduct.
4) Method: Fine grained instructions on how to conduct the task outlined above.

In [ ]:
PERSONA = """<PERSONA>
You are an experienced astrophysicist, and your task is to classify astronomical transients into Real or Bogus based on a given set of 3 images. You have seen thousands of astronomical images during your lifetime and you are very good at making this classification by looking at the images and following the instructions.
</PERSONA>"""

TASK = """<TASK>
Your task is to read the INSTRUCTIONS, look at the 3 images (New, Reference and Difference images) and classify if the source at the centre of the cutout and inside the red circle is a Real or Bogus astronomical transient. Provide your thought process to explain how you reasoned to provide the response. Respond in json format
</TASK>\n
"""

INSTRUCTIONS = """\n<INSTRUCTIONS>
**1. Purpose**
Help vet astronomical data for the Real/Bogus classification. The goal is for you to use your expertise to distinguish between real and bogus sources. 

**2. Information Provided**
You will be shown three astronomical image cutouts:
a) **New Image:** The newest image centered at the location of the suspected transient source. 
b) **Reference Image:** A reference image from the same telescope of the same part of the sky to be used for comparison. It shows if the source was already there in the past or not.
c) **Difference Image:** The residual image after the new and reference images are subtracted. Real sources should appear in this cutout as circular objects with only positive (white pixels) or only negative (black pixels) flux. 

**3. Criteria for Classification**
- **Real Source:** 
  - **Shape:** Circular shape at the center of the cutout with a visual extent of ~5-10 pixels, varying with focus conditions.
  - **Brightness:** Positive flux (white pixels) in either the new or reference image. Positive or negative flux in the Difference image. 
  - **Variability:** The source at the center can fade or brighten between the new and reference images, appearing as positive or negative in the Difference image.
  - **Presence:** The source may (dis)appear between the new and reference images. A source may also appear on top of an underlying source (e.g., supernova on a galaxy).

- **Bogus Source:** 
  - **Shape:** Non-circular shape (e.g., elongated). This includes irregular shapes, positive or negative, like streaks or lines caused by cosmic-rays, diffraction spikes and cross-talk.
  - **Brightness:** Negative flux (black pixels) at the center of the cutout in either the new or reference image. The source at the center can never be negative in the New or Reference image, only in the Differnece.
  - **Misalignment:** If the source in the New and Reference images is misaligned, it will show a Yin-Yang pattern (both white and black) in the Difference image.

**4. Additional Guidance** 
- **Contextual Information:** Focus on the source at the center of the cutouts inside the red circle, but consider nearby sources to diagnose potential problems.  
- **Examples:** Refer to provided visual examples of real and bogus sources to aid in identification. 
- **Judgment Criteria:** For ambiguous cases or borderline scenarios, consider the overall context and consistency with known characteristics of real and bogus sources.
</INSTRUCTIONS>"""


METHOD = """<METHOD>
1. **Focus on the Red Circle**: Start by examining the source located at the center of the cutout and inside the red circle. The images are prepared so that the source of interest is clearly marked for you to analyze.

2. **Analyze Each Image Individually**:
   - **New Image**: Check for the presence, shape, and brightness of the source in the new image.
   - **Reference Image**: Compare the source's properties in the reference image to those in the new image.
   - **Difference Image**: Observe the residuals that result from subtracting the reference image from the new image. Look for patterns (circular, positive/negative flux) that match characteristics of Real or Bogus sources.

3. **Evaluate Features**:
   - Examine the shape, brightness, and other relevant features (e.g., artifacts, misalignments) of the source in each image.
   - Determine if these features are consistent with a Real or Bogus classification based on the criteria provided in the instructions.

4. **Consider Relationships Between Images**:
   - Compare the new, reference, and difference images to understand any changes in the source over time.
   - Look for discrepancies or confirmations that might support or contradict a particular classification.

5. **Employ a Chain-of-Thought Reasoning**:
   - Clearly outline each observation you make and explain how it contributes to your decision-making process.
   - If you find any contradictions or ambiguous features, acknowledge them and provide reasoning for your final decision.

6. **Assign an Interest Score**:
   - After determining if the source is Real or Bogus, assign an appropriate interest score:
     - 'No interest' for Bogus sources.
     - 'Low interest' for variable transients.
     - 'High interest' for explosive transients.

7. **Prepare the Final Output in JSON Format**:
   - Format your response as a JSON object containing:
     - The classification ('Real' or 'Bogus').
     - An explanation detailing your thought process and observations.
     - The assigned interest score.

8. **Example Output**:
   - Refer to the provided examples to see the expected format and detail level of your response.
</METHOD>
"""

# Collapse the System Instructions into a single variable
stat_prompt = PERSONA + TASK + INSTRUCTIONS + METHOD

## Build the Prompt

In this part, we will build the prompt (the dynamic) part of each LLM call for each of the sample in the dataset. The process for building these examples would be as follows:

1) Download the raw image data (.npy files) and the labels (.csv) file associated with the images
2) Upload the labels to BigQuery
3) Seperate example set images (i.e. the few shot examples - 7 Real and 7 Bogus examples) that will be sent as a part of each LLM call. They will be stored under sample_indexes.
4) Convert the raw .npy files to pictures so that they can be sent to Gemini and store them under the data folder.
5) For each example in sample_indexes, define the expected output for class, explanation and interest_score.
6) Store the dynamic part of the prompt under the variable EXAMPLES.

In [ ]:
import pandas as pd
import numpy as np
import os

# Load the dataset
file_path_data = 'data/new_data.npy'
file_path_labels_csv = 'data/new_labels.csv'

triplets = np.load(file_path_data)

# Load labels from CSV file
labels_df = pd.read_csv(file_path_labels_csv)

# Upload Labels to BigQuery
labels_id = "MeerLICHT_labels_df"
labels_ref = bq_client.dataset(DATASET_ID).table(labels_id)
create_table_flag = if_tbl_exists(bq_client, labels_ref)

if create_table_flag != True:
    bq_client.load_table_from_dataframe(labels_df, labels_ref)

# Define sample indexes for example images
sample_indexes = [0, 1, 3, 4, 8, 48, 77, 592, 685, 1179, 1180, 1181, 1191, 1193, 3216]

# Save example pictures
for i in sample_indexes:
    save_picture(triplets, i, True)

# Identify valid indices by removing any with corrupt data
valid_indices = np.where(~np.isnan(triplets).any(axis=(1, 2, 3)))[0]

# Exclude sample_indexes from the prediction set
batch_index = np.setdiff1d(valid_indices, sample_indexes)

# Save prediction pictures
for t in batch_index:
    save_picture(triplets, t, False)

print(f"Saved {len(batch_index)} prediction pictures excluding examples.")

In [ ]:
## DESCRIPTION INDEX 0:
desc1 = {
  "class": "Bogus",
  "explanation": "In the New image, a diffraction spike is observed near the center. The Reference image also shows a diffraction spike at the same location. In the Difference image, a negative residual of the bright diffraction spike from the Reference image is clearly visible. The consistent presence of diffraction spikes in all three images, without a clear circular source, confirms that this is a bogus source.",
  "interest_score": "No interest"
}

## DESCRIPTION INDEX 1:
desc2 = {
  "class": "Bogus",
  "explanation": "In the New image, a negative elongated artifact is present at the center. The Reference image does not show any source at the same location. In the Difference image, the same negative artifact appears, which results from the negative clump of pixels in the New image. Since a real source cannot be negative in the New image, this is classified as a bogus source.",
  "interest_score": "No interest"
}


## DESCRIPTION INDEX 3:
desc3 = {
  "class": "Bogus",
  "explanation": "In the New image, the source appears as a streak of several bright pixels and is not circular. The Reference image shows no source at the same location. The Difference image shows the same streak of pixels as in the New image. The sharp, streak-like appearance in the New image suggests that this is most likely a cosmic ray rather than a real source.",
  "interest_score": "No interest"
}

## DESCRIPTION INDEX 4:
desc4 = {
  "class": "Bogus",
  "explanation": "The New image does not have any source at the centre of the cutout. The Reference image shows a source appearing as a streak of a few bright pixels, which is not circular. The difference image shows the negative residual of the same streak present in the Reference image. This is too sharp to be a real source and is likely a cosmic ray that was not flagged during the creation of the Reference image.",
  "interest_score": "No interest"
}

## DESCRIPTION INDEX 8:
desc5= {
  "class": "Bogus",
  "explanation": "No source is present in the New image. In the Reference image, a source appears as a negative circular object. The Difference image presents a faint positive residual of the source in the Reference image. A source cannot be negative in the Reference image, this is not a real source.",
  "interest_score": "No interest"
}

## DESCRIPTION INDEX 48:
desc6 = {
  "class": "Bogus",
  "explanation": "The New image does not have any source at the centre of the cutout. In the Reference image, the source appears very elongated. The Difference shows the same negative elongated source, supporting the conclusion that it a bogus source.",
  "interest_score": "No interest"
}

## DESCRIPTION INDEX 77:
desc7 = {
  "class": "Bogus",
  "explanation": "In the New image, a small elongated source is visible, surrounded by several other sources. The Reference image shows no source at the same location, but it does show all the other sources. In the Difference image, the residual is positive but its elongation confirms this is a bogus source.",
  "interest_score": "No interest"
}


## DESCRIPTION INDEX 592:
desc8 = {
  "class": "Bogus",
  "explanation": "The New image shows a diffuse source at the center, aligned with a 45-degree diffraction spike from a bright source at the corner of the cutout. The Reference image also shows a diffraction spike and a similar blob. The Difference image displays a positive blob, indicating it is an artifact caused by the diffraction spike, which can produce blobs or irregular shapes.",
  "interest_score": "No interest"
}

## DESCRIPTION INDEX 685:
desc9 = {
  "class": "Bogus",
  "explanation": "The New image shows no source at the center. The Reference image shows a faint positive trail cutting diagonally across the image with a circular source at the center, likely caused by a blinking object like an airplane or satellite. The Difference image displays both the trail and a negative blob, confirming the source is a non-astronomical artifact.",
  "interest_score": "No interest"
}


## DESCRIPTION INDEX 1179:
desc10 = {
  "class": "Real",
  "explanation": "The New image shows a source at the center. The Reference image also shows the same source in the same location. The Difference image has a positive residual, indicating the source has brightened. This pattern suggests the source is a real variable star.",
  "interest_score": "Low interest"
}


## DESCRIPTION INDEX 1180:
desc11 = {
  "class": "Real",
  "explanation": "The New image shows a source at the center. The Reference image also shows the same source in the same location. The Difference image has a negative residual, indicating the source has dimmed. This pattern suggests the source is a real variable star.",
  "interest_score": "Low interest"
}


## DESCRIPTION INDEX 1181:
desc12 = {
  "class": "Real",
  "explanation": "The New image shows no source at the center. The Reference image shows a circular source in the same location. The Difference image displays a negative circular residual, consistent with a transient that has disappeared.",
  "interest_score": "High interest"
}


## DESCRIPTION INDEX 1191:
desc13 = {
  "class": "Real",
  "explanation": "The New image shows a bright circular source at the center. The Reference image shows no source in the same location. The Difference image displays a positive circular residual, indicating a real explosive transient.",
  "interest_score": "High interest"
}

## DESCRIPTION INDEX 1193:
desc14 = {
  "class": "Real",
  "explanation": "The New image shows a source at the center. The Reference image also shows the same source in the same location. The Difference image displays a positive residual, indicating the source has brightened. A cosmic ray artifact is visible to the left, but the central source is unaffected and remains a valid transient.",
  "interest_score": "Low interest"
}

## DESCRIPTION INDEX 3216:
desc15 = {
  "class": "Real",
  "explanation": "The New image shows a source at the center, superimposed on a diffuse galaxy. The Reference image displays the galaxy but no source at the same location. The Difference image reveals a faint, positive circular feature, consistent with a supernova emerging within the galaxy.",
  "interest_score": "High interest"
}

descriptions = [str(desc1), str(desc2), str(desc3), str(desc4), str(desc5), str(desc6),str(desc7), str(desc8), str(desc9), str(desc10), str(desc11), str(desc12), str(desc13), str(desc14), str(desc15)]

### Write the examples used in a readable format to be saved as a txt file for tracebility
example_description = list(zip(["DESCRIPTION INDEX: " + str(x) for x in sample_indexes], descriptions))

In [ ]:
# Report 15 examples for the dynamic prompt
EXAMPLES = ["<EXAMPLES>\n"]
for i in range(len(sample_indexes)):
    
    str_EX = f"""Example {i+1}:
    """
    all_list = create_ex(sample_indexes[i], True)
    all_list.insert(0, str_EX)
    all_list.append(descriptions[i])
    all
    for k in all_list:
        EXAMPLES.append(k)
EXAMPLES.append("\n</EXAMPLES>\n")

# 3. Running the batch run

After creating the system instructions and the prompt for each example, we will first create the experiment variables for tracebility and then initiate the batch process with the function ```build_run_batch``` function from ```helper_functions.py```

```build_run_batch``` does the following:
1. Constructs input and output table names based on the project ID and formatted datetime.
2. Defines the table schema with 'request' (JSON) and 'index_no' (INTEGER) fields.
3. Creates the input table in BigQuery if it doesn't exist.
4. For index item in the batch_index:
    - Constructs a dynamic prompt using the provided examples and the current index.
    - Creates a batch data dataframe using the static prompt, dynamic prompt, and specified parameters.
    - Uploads the dataframe to a GCS bucket
    - Creates a Big query table using the data stored in GCS bucket. 
5. Generates a request.json file for batch processing.
6. Sends the batch prediction job to the specified project.
7. Waits until the batch prediction job concludes.
8. Generate a Big Query table that processes the ```BatchPredictionJob```
9. Download the table and exports as a pandas dataframe


In [ ]:
# Start logging the experiment

## Prepare the variables
timestamp = datetime.datetime.now()
formatted_datetime = timestamp.strftime('%Y%m%d%H%M')

## Log the experiments variables
### Create the run name with timestamp
run_name = "run" + formatted_datetime
DESCRIPTION = """This is the final run to be pushed to the repo highlighting reproducibility""" # @param {type:"string"}
MODEL = "gemini-1.5-pro-002" # @param [gemini-1.5-pro-001", "gemini-1.5-flash-001", "gemini-1.0-pro-002"]
TEMPERATURE = .8 # @param {type:"slider", min:0, max:2, step:0.1}
TOP_P = 1 # @param {type:"slider", min:0, max:1, step:0.05}
PROMPT_FILE = save_prompt(stat_prompt + '\n'.join([a + "\n" + b + "\n" for (a,b) in example_description]), run_name)

# Build the experimentation variables
exp_vars = build_experiment_vars(description=DESCRIPTION,prompt=PROMPT_FILE, model=MODEL, temperature=TEMPERATURE, top_p=TOP_P)
# # Start the run
aiplatform.start_run(run_name)
# # Log the experiment variables
aiplatform.log_params(exp_vars)

In [ ]:
# Build and then the batch run requests
pred_df = build_run_batch(bq_client, batch_index, labels_ref, PROJECT_ID, DATASET_ID, run_name, MODEL, stat_prompt, EXAMPLES, TEMPERATURE, TOP_P)

# 4. Analyze the results and conclude the experiment

Once the batch run is completed the results are stored in pred_df dataframe. We will next evaluate the performance of the binary classification model by generating a confusion matrix and calculating key performance metrics.

The evaluation will be conducted via:

1.  **Visualize Model Performance:** Create and display a confusion matrix to visually represent the model's performance in classifying instances into two categories (likely "Real" and "Bogus").
2.  **Calculate Performance Metrics:** Compute essential performance metrics (Accuracy, Precision, and Recall) based on the confusion matrix.

Finally we will conclude the code by logging the metrics and ending the experiment run.


In [ ]:
# Generate the confusion matrix for the results
plt.clf()
# Adding this line to make sure that the ground truth data is reflected correctly in the pred_df dataframe
pred_df = pd.merge(pred_df, labels_df[['index_no', 'label']], on='index_no', how='left')
pred_cleaned = pred_df[(pred_df.predicted == "Real") | (pred_df.predicted == "Bogus")]
cm = confusion_matrix(pred_cleaned.label, pred_cleaned.predicted)
disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot()
plt.show()
# Calculate TP, TN, FP, FN
TP = cm[0][0]
TN = cm[1][1]
FP = cm[0][1]
FN = cm[1][0]
# Print the metrics
print(f"Accuracy is {(TN+TP)/(TN+TP+FP+FN)}")
print(f"Precision is {TP/(TP+FP)}")
print(f"Recall is {TP/(TP+FN)}")

#Save the data frame for reproducibility 
pred_df.to_csv("data/predictions_results.csv", mode="x")

In [13]:
# Log the KPI and conclude the experiment
aiplatform.log_metrics(build_experiment_vars(accuracy=(TN+TP)/(TN+TP+FP+FN), precision=TP/(TP+FP), recall=TP/(TP+FN)))
aiplatform.end_run()

# Cleaning up

In this exercise we showed how to use Gemini models to distinguish between real astrophysical signals (e.g., explosive events, variable stars) and bogus imaging artifacts. This is the first demonstration of a successful application of an LLM to imaging data from optical transient surveys. 

In this part we will be cleaning after resources used above.

In [14]:
# Clean up the resources. 

# Delete the BigQeury Tables
bq_client.delete_table(labels_ref)
bq_client.delete_table(f"{PROJECT_ID}.{DATASET_ID}.{run_name}")